In [ ]:
import os
import random
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix
)
from tqdm import tqdm
from transformers import (
    EfficientFormerImageProcessor,
    EfficientFormerForImageClassification,
)
from torch import amp   # ★ 최신 AMP


# =====================================================
# 1. SEED
# =====================================================
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = False

set_seed(42)
device = "cuda" if torch.cuda.is_available() else "cpu"


# =====================================================
# 2. DATASET
# =====================================================
class EmotionBinaryDataset(Dataset):
    def __init__(self, root_dir, processor):
        self.root_dir = root_dir
        self.processor = processor
        self.samples = []

        valid_folders = ["happy", "anger", "panic", "sadness"]

        for cls in valid_folders:
            folder_path = os.path.join(root_dir, cls)
            if not os.path.exists(folder_path):
                continue

            label = 1 if cls == "happy" else 0

            for file in os.listdir(folder_path):
                if file.lower().endswith(("jpg", "jpeg", "png")):
                    self.samples.append(
                        (os.path.join(folder_path, file), label)
                    )

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        img = Image.open(path).convert("RGB")

        processed = self.processor(images=img, return_tensors="pt")
        pixel_values = processed["pixel_values"].squeeze(0)

        return pixel_values, torch.tensor(label)


# =====================================================
# 3. LOAD Model
# =====================================================
processor = EfficientFormerImageProcessor.from_pretrained(
    "snap-research/efficientformer-l1-300"
)

model = EfficientFormerForImageClassification.from_pretrained(
    "snap-research/efficientformer-l1-300",
    num_labels=2,
    ignore_mismatched_sizes=True
).to(device)

scaler = amp.GradScaler(device="cuda")   # ★ 최신 문법


# =====================================================
# 4. PATHS
# =====================================================
train_root = "/workspace/user4/cropped/train"
val_root   = "/workspace/user4/cropped/val"

train_dataset = EmotionBinaryDataset(train_root, processor)
val_dataset   = EmotionBinaryDataset(val_root, processor)

train_loader = DataLoader(
    train_dataset, batch_size=32, shuffle=True,
    num_workers=8, pin_memory=True, persistent_workers=True
)
val_loader = DataLoader(
    val_dataset, batch_size=32, shuffle=False,
    num_workers=8, pin_memory=True, persistent_workers=True
)


# =====================================================
# 5. Optimizer
# =====================================================
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5)


# =====================================================
# 6. TRAINING LOOP
# =====================================================
EPOCHS = 10

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0.0

    pbar = tqdm(train_loader, desc=f"[Epoch {epoch+1}] Training")

    for pixel_values, labels in pbar:
        pixel_values = pixel_values.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)

        optimizer.zero_grad()

        # ★ 최신 AMP 스타일
        with amp.autocast("cuda"):
            outputs = model(pixel_values=pixel_values, labels=labels)
            loss = outputs.loss

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()
        pbar.set_postfix({"loss": loss.item()})

    avg_loss = total_loss / len(train_loader)
    print(f"\n[Epoch {epoch+1}] Train Loss: {avg_loss:.4f}")

    # -------------------------
    # Validation
    # -------------------------
    model.eval()
    preds, gts = [], []

    with torch.no_grad():
        for pixel_values, labels in tqdm(val_loader, desc="Validating"):
            pixel_values = pixel_values.to(device, non_blocking=True)

            with amp.autocast("cuda"):
                outputs = model(pixel_values=pixel_values)
                pred = outputs.logits.argmax(dim=1).cpu()

            preds.extend(pred.tolist())
            gts.extend(labels.tolist())

    acc = accuracy_score(gts, preds)
    prec = precision_score(gts, preds)
    rec  = recall_score(gts, preds)
    f1   = f1_score(gts, preds)
    cm   = confusion_matrix(gts, preds)

    print("\n========== METRICS ==========")
    print(f"Accuracy  : {acc:.4f}")
    print(f"Precision : {prec:.4f}")
    print(f"Recall    : {rec:.4f}")
    print(f"F1-Score  : {f1:.4f}")
    print("Confusion Matrix:")
    print(cm)
    print("============================\n")


from sklearn.metrics import classification_report

# =====================================================
# FINAL SUMMARY + Classification Report
# =====================================================

print("\n--- Final Summary (Validation Set) ---")
print(f"Train Loss: {avg_loss:.4f}")      # 마지막 epoch train loss
print(f"Val Loss  : {f1:.4f}")            # ★ 필요 시 val_loss 변수로 교체 가능

from sklearn.metrics import classification_report

target_names = ["others", "happy"]  # 0 → others, 1 → happy

report = classification_report(
    gts,
    preds,
    target_names=target_names,
    digits=4
)

print("\n--- Classification Report (Validation Set) ---")
print(report)


Some weights of EfficientFormerForImageClassification were not initialized from the model checkpoint at snap-research/efficientformer-l1-300 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 448]) in the checkpoint and torch.Size([2, 448]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
[Epoch 1] Training: 100%|██████████| 298/298 [00:37<00:00,  7.92it/s, loss=0.0924]



[Epoch 1] Train Loss: 0.3161


Validating: 100%|██████████| 36/36 [00:05<00:00,  6.61it/s]



========== METRICS ==========
Accuracy  : 0.9321
Precision : 0.8166
Recall    : 0.9532
F1-Score  : 0.8796
Confusion Matrix:
[[786  64]
 [ 14 285]]



[Epoch 2] Training: 100%|██████████| 298/298 [00:27<00:00, 10.74it/s, loss=0.0224] 


...


========== METRICS ==========
Accuracy  : 0.9547
Precision : 0.8754
Recall    : 0.9632
F1-Score  : 0.9172
Confusion Matrix:
[[809  41]
 [ 11 288]]



[Epoch 10] Training: 100%|██████████| 298/298 [00:27<00:00, 10.82it/s, loss=0.000412]



[Epoch 10] Train Loss: 0.0006


Validating: 100%|██████████| 36/36 [00:03<00:00,  9.43it/s]


========== METRICS ==========
Accuracy  : 0.9547
Precision : 0.8777
Recall    : 0.9599
F1-Score  : 0.9169
Confusion Matrix:
[[810  40]
 [ 12 287]]


--- Classification Report (Validation Set) ---
                 precision    recall    f1-score    support
others              0.9854    0.9529      0.9689       850.0
happy               0.8777    0.9599      0.9169       299.0

accuracy                               0.9547        1149
macro avg                 0.9315    0.9564      0.9429        1149
weighted avg             0.9574    0.9547      0.9554        1149


In [ ]:
# =====================================================
# 6. TRAINING LOOP
# =====================================================
EPOCHS = 10
criterion = torch.nn.CrossEntropyLoss()   # ★ 추가됨: val_loss 계산용

for epoch in range(EPOCHS):
    model.train()
    total_train_loss = 0.0    # ★ train loss 합산

    pbar = tqdm(train_loader, desc=f"[Epoch {epoch+1}] Training")

    for pixel_values, labels in pbar:
        pixel_values = pixel_values.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)

        optimizer.zero_grad()

        # AMP
        with amp.autocast("cuda"):
            outputs = model(pixel_values=pixel_values, labels=labels)
            loss = outputs.loss

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_train_loss += loss.item()  # ★ train loss 기록
        pbar.set_postfix({"loss": loss.item()})

    avg_train_loss = total_train_loss / len(train_loader)  # ★ train loss 평균
    print(f"\n[Epoch {epoch+1}] Train Loss: {avg_train_loss:.4f}")

    # -------------------------
    # Validation
    # -------------------------
    model.eval()
    preds, gts = [], []
    total_val_loss = 0.0   # ★ val loss 합산

    with torch.no_grad():
        for pixel_values, labels in tqdm(val_loader, desc="Validating"):
            pixel_values = pixel_values.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)

            with amp.autocast("cuda"):
                outputs = model(pixel_values=pixel_values)
                logits = outputs.logits
                loss = criterion(logits, labels)    # ★ val loss 계산

            total_val_loss += loss.item()          # ★ val loss 누적

            pred = logits.argmax(dim=1).cpu()
            preds.extend(pred.tolist())
            gts.extend(labels.tolist())

    avg_val_loss = total_val_loss / len(val_loader)  # ★ val loss 평균 계산

    # ★ METRICS
    acc = accuracy_score(gts, preds)
    prec = precision_score(gts, preds, average='macro')
    rec  = recall_score(gts, preds, average='macro')
    f1   = f1_score(gts, preds, average='macro')
    cm   = confusion_matrix(gts, preds)

    print("\n========== METRICS ==========")
    print(f"Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")  # ★ 출력
    print(f"Accuracy  : {acc:.4f}")
    print(f"Precision : {prec:.4f}")
    print(f"Recall    : {rec:.4f}")
    print(f"F1-Score  : {f1:.4f}")
    print("Confusion Matrix:")
    print(cm)
    print("============================\n")


# =====================================================
# FINAL Classification Report
# =====================================================
model.eval()
preds, gts = [], []

with torch.no_grad():
    for pixel_values, labels in tqdm(val_loader, desc="Validating"):
        pixel_values = pixel_values.to(device, non_blocking=True)

        with amp.autocast("cuda"):
            outputs = model(pixel_values=pixel_values)
            pred = outputs.logits.argmax(dim=1).cpu()

        preds.extend(pred.tolist())
        gts.extend(labels.tolist())

target_names = ["happy", "anger", "panic", "sadness"]

report = classification_report(
    gts,
    preds,
    target_names=target_names,
    digits=4
)

print("\n--- Classification Report (Validation Set) ---")
print(report)


Some weights of EfficientFormerForImageClassification were not initialized from the model checkpoint at snap-research/efficientformer-l1-300 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 448]) in the checkpoint and torch.Size([4, 448]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([4]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
[Epoch 1] Training: 100%|██████████| 183/183 [02:23<00:00,  1.28it/s, loss=1.18] 



[Epoch 1] Train Loss: 1.2881


Validating: 100%|██████████| 36/36 [00:30<00:00,  1.20it/s]



========== METRICS ==========
Accuracy  : 0.5491
Precision : 0.5326
Recall    : 0.5506
F1-Score  : 0.5203
Confusion Matrix:
[[203  23  30  43]
 [ 64  49  98  80]
 [ 17  24 209  30]
 [ 48  13  49 171]]



[Epoch 2] Training: 100%|██████████| 183/183 [02:16<00:00,  1.34it/s, loss=0.842]


...


[Epoch 10] Train Loss: 0.0272


Validating: 100%|██████████| 36/36 [00:26<00:00,  1.36it/s]



========== METRICS ==========
Accuracy  : 0.7359
Precision : 0.7354
Recall    : 0.7345
F1-Score  : 0.7349
Confusion Matrix:
[[264  13  11  11]
 [ 13 183  41  54]
 [ 13  50 203  14]
 [  9  53  22 197]]



Validating: 100%|██████████| 36/36 [00:25<00:00,  1.38it/s]


--- Classification Report (Validation Set) ---
              precision    recall  f1-score   support

       happy     0.8829    0.8829    0.8829       299
       anger     0.6120    0.6289    0.6203       291
       panic     0.7329    0.7250    0.7289       280
     sadness     0.7138    0.7011    0.7074       281

    accuracy                         0.7359      1151
   macro avg     0.7354    0.7345    0.7349      1151
weighted avg     0.7366    0.7359    0.7362      1151

